In [1]:
import numpy as np

import solver as sv

def multimodewave(L, modes):
    with sv.Solver() as S:
        for mode,n in modes.items():
            sv.Waveguide(L,n=n).expand_mode([mode]).put()
        sv.raise_pins()
    return S

modes = {
    'TE0' : 3.5,
    'TE1' : 3.4,
    'TM0' : 3.3,
    'TM1' : 3.2,
}

WG = multimodewave(10.0, modes)
WG.solve(wl=1.55).S2PD(func=np.abs)


with sv.Solver() as S:
    wg1 = WG.put()
    wg2 = WG.put()
    sv.connect_all(wg1,'b0',wg2,'a0')
    
S.show_connections()

Connection of solver:                 Solver object (id=140329337307032))
(Structure (id=140329336862312) containing Solver object (id=140329336861472), b0_TM1) <--> (Structure (id=140329336863936) containing Solver object (id=140329336862424), a0_TM1)
(Structure (id=140329336862312) containing Solver object (id=140329336861472), b0_TE1) <--> (Structure (id=140329336863936) containing Solver object (id=140329336862424), a0_TE1)
(Structure (id=140329336862312) containing Solver object (id=140329336861472), b0_TM0) <--> (Structure (id=140329336863936) containing Solver object (id=140329336862424), a0_TM0)
(Structure (id=140329336862312) containing Solver object (id=140329336861472), b0_TE0) <--> (Structure (id=140329336863936) containing Solver object (id=140329336862424), a0_TE0)



# Multiple modes and polrizations

GenSol is not really equipped for dealing with multiple modes at the same port. This because, even if two or more modes may be traveling through one single physical port, from the point of view of the scattering matrix they are represented by two distinct port. 

So, in order to deal with multiple modes, no real modification to the code is needed. Simply, each combination of port/mode is mapped to a single entry in the scattering matrix, and the fact that multiple modes may correspond to to the same physical port is left as an interpretation exercise. 

However, it may be convenient to reflect this mapping in a clever naming of the pins. One particularly useful is 'basename_modename'. This convention is not mandatory to use the code in itself, but using it allows for the usage a some helper functions that can be useful to speed up model definition. In addition, it is the convention assumed by the Nazca integration. 



In [2]:
import numpy as np
import matplotlib.pyplot as plt

import solver as sv

## Building multiple modes components from single mode ones: expand_mode
The first useful function is the `expand_mode`. It takes a list of string (names of the modes) and creates, for each port of the base models, a set of ports with names `originalname_modename`. The scattering matrix of the original models is copied to the new modes, so the behavior of all the modes will be identical no interaction between them is possible in this wave. 

### Basic Usage

Here is an example on a single waveguide:

In [3]:
wave = sv.Waveguide(10.0, n=1.0)
wave.S2PD()

,a0,b0
a0,0.000000+0.000000j,1.000000-0.000000j
b0,1.000000-0.000000j,0.000000+0.000000j


In [4]:
wave.expand_mode(['TE0','TM0', 'TE1'])
wave.S2PD()

,a0_TE0,a0_TE1,a0_TM0,b0_TE0,b0_TE1,b0_TM0
a0_TE0,0.000000+0.000000j,0.000000+0.000000j,0.000000+0.000000j,1.000000-0.000000j,0.000000+0.000000j,0.000000+0.000000j
a0_TE1,0.000000+0.000000j,0.000000+0.000000j,0.000000+0.000000j,0.000000+0.000000j,1.000000-0.000000j,0.000000+0.000000j
a0_TM0,0.000000+0.000000j,0.000000+0.000000j,0.000000+0.000000j,0.000000+0.000000j,0.000000+0.000000j,1.000000-0.000000j
b0_TE0,1.000000-0.000000j,0.000000+0.000000j,0.000000+0.000000j,0.000000+0.000000j,0.000000+0.000000j,0.000000+0.000000j
b0_TE1,0.000000+0.000000j,1.000000-0.000000j,0.000000+0.000000j,0.000000+0.000000j,0.000000+0.000000j,0.000000+0.000000j
b0_TM0,0.000000+0.000000j,0.000000+0.000000j,1.000000-0.000000j,0.000000+0.000000j,0.000000+0.000000j,0.000000+0.000000j


As can be seen, building block with a arbitrary numbers of modes can be generated quite easily. 

Oss: `expand_mode` does change the instance of the class on which is called. Be careful when using in Spyder of Jupyter Notebook.

It can be used on models with any number of ports. For example, here is a an example a a two modes beam splitter.

In [5]:
sv.GeneralBeamSplitter().expand_mode(['TE', 'TM']).S2PD(func=np.abs)

,a0_TE,a0_TM,a1_TE,a1_TM,b0_TE,b0_TM,b1_TE,b1_TM
a0_TE,0.000000,0.000000,0.000000,0.000000,0.707107,0.000000,0.707107,0.000000
a0_TM,0.000000,0.000000,0.000000,0.000000,0.000000,0.707107,0.000000,0.707107
a1_TE,0.000000,0.000000,0.000000,0.000000,0.707107,0.000000,0.707107,0.000000
a1_TM,0.000000,0.000000,0.000000,0.000000,0.000000,0.707107,0.000000,0.707107
b0_TE,0.707107,0.000000,0.707107,0.000000,0.000000,0.000000,0.000000,0.000000
b0_TM,0.000000,0.707107,0.000000,0.707107,0.000000,0.000000,0.000000,0.000000
b1_TE,0.707107,0.000000,0.707107,0.000000,0.000000,0.000000,0.000000,0.000000
b1_TM,0.000000,0.707107,0.000000,0.707107,0.000000,0.000000,0.000000,0.000000


### Tips and Tricks
The `expand_mode` function can be useful even when the aim is to define different behaviors for different modes. It is simply a matter of building the model for for each mode, rename the pins with `expand_pol`, wrap everything into a `Solver` and use `raise_pins` for quick mapping. 

Here is, for example, how to build a Beam Splitter with different ratio in TE (75/25) and TM (25/75):

In [6]:
with sv.Solver() as MultiBM:
    sv.GeneralBeamSplitter(ratio=0.75).expand_mode(['TE']).put()
    sv.GeneralBeamSplitter(ratio=0.25).expand_mode(['TM']).put()
    sv.raise_pins()
    
MultiBM.solve(wl=1.55).S2PD(np.abs)
    

,a0_TE,a0_TM,a1_TE,a1_TM,b0_TE,b0_TM,b1_TE,b1_TM
a0_TE,0.000000,0.000000,0.000000,0.000000,0.500000,0.000000,0.866025,0.000000
a0_TM,0.000000,0.000000,0.000000,0.000000,0.000000,0.866025,0.000000,0.500000
a1_TE,0.000000,0.000000,0.000000,0.000000,0.866025,0.000000,0.500000,0.000000
a1_TM,0.000000,0.000000,0.000000,0.000000,0.000000,0.500000,0.000000,0.866025
b0_TE,0.500000,0.000000,0.866025,0.000000,0.000000,0.000000,0.000000,0.000000
b0_TM,0.000000,0.866025,0.000000,0.500000,0.000000,0.000000,0.000000,0.000000
b1_TE,0.866025,0.000000,0.500000,0.000000,0.000000,0.000000,0.000000,0.000000
b1_TM,0.000000,0.500000,0.000000,0.866025,0.000000,0.000000,0.000000,0.000000


## Fast connections: connect_all
When connecting structures featuring many modes, it could be annoying to type a `connect` statement for each mode, since all the modes related to the same physical port should be connected together. For this reason, the function `connect_all` is provided, with connects all the common modes of two pin basenames between two structures:

In [8]:
WG = sv.Waveguide(50.0,1.0).expand_mode(['TE0','TM0', 'TE1'])

with sv.Solver() as S:
    wg1 = WG.put()
    wg2 = WG.put()
    sv.connect_all(wg1,'b0',wg2,'a0')
    sv.raise_pins()
    
S.show_free_pins()
S.show_connections()

        

Free pins of solver:                 Solver object (id=140329303759224))
(Structure (id=140329303812416) containing Model of waveguide of lenght 50.000 and index 1.000 (id=140329303812360), a0_TE0) --> a0_TE0
(Structure (id=140329303812416) containing Model of waveguide of lenght 50.000 and index 1.000 (id=140329303812360), a0_TM0) --> a0_TM0
(Structure (id=140329303812416) containing Model of waveguide of lenght 50.000 and index 1.000 (id=140329303812360), a0_TE1) --> a0_TE1
(Structure (id=140329303812640) containing Model of waveguide of lenght 50.000 and index 1.000 (id=140329303812584), b0_TE0) --> b0_TE0
(Structure (id=140329303812640) containing Model of waveguide of lenght 50.000 and index 1.000 (id=140329303812584), b0_TM0) --> b0_TM0
(Structure (id=140329303812640) containing Model of waveguide of lenght 50.000 and index 1.000 (id=140329303812584), b0_TE1) --> b0_TE1

Connection of solver:                 Solver object (id=140329303759224))
(Structure (id=140329303812416) cont

As can be seen by the output of the previous commands, all the internal pins are connected correctly using only one statement. 

Please note that `connect_all` only connects the common modes between the pins. If a mode is missing in one, it is left unconnected:

In [9]:
WG1 = sv.Waveguide(50.0,1.0).expand_mode(['TE0','TM0', 'TE1'])
WG2 = sv.Waveguide(50.0,1.0).expand_mode(['TE0','TM0', 'TM1'])

with sv.Solver() as S:
    wg1 = WG1.put()
    wg2 = WG2.put()
    sv.connect_all(wg1,'b0',wg2,'a0')
    sv.raise_pins()
    
S.show_free_pins()
S.show_connections()

ERROR:solver:b0 in Structure (id=140329303335936) containing Model of waveguide of lenght 50.000 and index 1.000 (id=140329303335880) and a0 in Structure (id=140329303336104) containing Model of waveguide of lenght 50.000 and index 1.000 (id=140329303336048) and have differents modes: only matching modes connected


Free pins of solver:                 Solver object (id=140329303335096))
(Structure (id=140329303335936) containing Model of waveguide of lenght 50.000 and index 1.000 (id=140329303335880), a0_TE0) --> a0_TE0
(Structure (id=140329303335936) containing Model of waveguide of lenght 50.000 and index 1.000 (id=140329303335880), a0_TM0) --> a0_TM0
(Structure (id=140329303335936) containing Model of waveguide of lenght 50.000 and index 1.000 (id=140329303335880), a0_TE1) --> a0_TE1
(Structure (id=140329303335936) containing Model of waveguide of lenght 50.000 and index 1.000 (id=140329303335880), b0_TE1) --> b0_TE1
(Structure (id=140329303336104) containing Model of waveguide of lenght 50.000 and index 1.000 (id=140329303336048), a0_TM1) --> a0_TM1
(Structure (id=140329303336104) containing Model of waveguide of lenght 50.000 and index 1.000 (id=140329303336048), b0_TE0) --> b0_TE0
(Structure (id=140329303336104) containing Model of waveguide of lenght 50.000 and index 1.000 (id=140329303336